In [ ]:
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.initializers import Constant
from keras.layers import Add, Activation
from keras.layers.convolutional import Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Input, Model
from keras.applications.vgg16 import VGG16
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from losses import binary_cross
import keras.backend as K
import metrics
from dataset import quick_dataset, load_images, normalize, per_chan_avg, apply_masks
from cv2 import bitwise_and
import nn_models
%matplotlib inline

# Pipeline experiments
Goal: To evaluate how well the network performs when trained to count.  Hypothesis is that the counting pipeline will perform the best on images which are more sparsely populated, and will count better using the segmented images.
The following conditions
* Small cells
* Large cells

Compare the following
* Segmentation quality
* Plaque counts

We are comparing 3 neural networks
1. Segmentation network trained on small cells
2. Segmentation network trained on large cells
3. Segmentation network trained on both large and small cells
* Compare mean IoU

Counting Comparison
1. Count network trained using cells isolated from original images (using the segmentation network with the best mean IoU)
2. Count network trained using the segmented images (training with the blobs instead of isolated cells)
* Compare the counts achieved by comparing:
    * Plot the counts, mean, and standard deviations
    * Evaluation of counts with mean, and standard deviation, for the individual counts
    * Identify a threshold, see if counting is better/worse at higher or lower counts

### Import the datasets
* Set 1: Small cells (training set [0:480], Validation set [0:120])
* Set 2: large cells (training set [480:960], Validation set [120:])
* Set 3: Combined cells set(training set[0:], validation set [0:])

In [ ]:
# Get the filename metadata and set the paths of the images and ground truth files
train_meta = pd.read_csv('training_set.csv')
val_meta = pd.read_csv('validation_set.csv')
gt_path = './BBBC005_v1_ground_truth'
img_path = './BBBC005_v1_images'

In [ ]:
# Get the small images for a dataset
tx_sm = load_images(img_names=train_meta.filename[:480], file_path=img_path, preprocess='image', target_size=(224,224,3))
ty_sm  = load_images(img_names=train_meta.filename[:480], file_path=gt_path, preprocess='gt', target_size=(224,224))
vx_sm = load_images(img_names=val_meta.filename[:120], file_path=img_path, preprocess='image', target_size=(224,224,3))
vy_sm = load_images(img_names=val_meta.filename[:120], file_path=gt_path, preprocess='gt', target_size=(224,224))

In [ ]:
# Get the large images for a dataset
tx_lg = load_images(img_names=train_meta.filename[480:], file_path=img_path, preprocess='image', target_size=(224,224,3))
ty_lg  = load_images(img_names=train_meta.filename[480:], file_path=gt_path, preprocess='gt', target_size=(224,224))
vx_lg = load_images(img_names=val_meta.filename[120:], file_path=img_path, preprocess='image', target_size=(224,224,3))
vy_lg = load_images(img_names=val_meta.filename[120:], file_path=gt_path, preprocess='gt', target_size=(224,224))

In [ ]:
# Get both the large and small images for a dataset
tx_full = load_images(img_names=train_meta.filename[0:], file_path=img_path, preprocess='image', target_size=(224,224,3))
ty_full  = load_images(img_names=train_meta.filename[0:], file_path=gt_path, preprocess='gt', target_size=(224,224))
vx_full = load_images(img_names=val_meta.filename[0:], file_path=img_path, preprocess='image', target_size=(224,224,3))
vy_full = load_images(img_names=val_meta.filename[0:], file_path=gt_path, preprocess='gt', target_size=(224,224))

In [ ]:
def get_counts(filenames):
    count = []
    for f in filenames:
        count.append(int(str.split(str.split(f, '_')[2], 'C')[1]))
    return count
train_counts_sm = get_counts(train_meta.filename[:480])
val_counts_sm = get_counts(val_meta.filename[:120])
train_counts_lg = get_counts(train_meta.filename[480:])
val_counts_lg = get_counts(val_meta.filename[120:])
train_counts_full = get_counts(train_meta.filename)
val_counts_full = get_counts(val_meta.filename)

In [ ]:
# Normalize the small datasets
avgs = per_chan_avg(tx_sm)
normalize(tx_sm, avgs)
normalize(vx_sm, avgs)

In [ ]:
# Normalize the large datasets
avgs = per_chan_avg(tx_lg)
normalize(tx_lg, avgs)
normalize(vx_lg, avgs)

In [ ]:
# Normalize the full datasets
avgs = per_chan_avg(tx_full)
normalize(tx_full, avgs)
normalize(vx_full, avgs)

In [ ]:
im_num = -1
fig, a = plt.subplots(1, 6)
a[0].imshow(tx_sm[im_num,:,:,2])
a[1].imshow(ty_sm[im_num,:,:,0])
a[2].imshow(tx_lg[im_num,:,:,2])
a[3].imshow(ty_lg[im_num,:,:,0])
a[4].imshow(tx_full[im_num,:,:,2])
a[5].imshow(ty_full[im_num,:,:,0])
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.tight_layout()

## Segmentation (Blob Detection) Network Comparison
#### Train the network on the small cells

In [ ]:
fcn_sm = nn_models.build_fcn_bilinear_8s(nb_classes=1)

In [ ]:
NB_EPOCHS = 1000
OPTIMIZER = Adam(lr=0.001)
LOSS = binary_cross # Binary cross entropy
VERBOSE = 1
BATCH_SIZE = 1

# Create a callback function to save the most accurate version of our model following each epoch, to stop if learning stops, and
# to reduce the learning rate if loss fails to decrease.
CALLBACKS = [ModelCheckpoint('mod/fcn8s_weights_sm.hdf5', monitor='loss', verbose=1, 
                           save_best_only=True, save_weights_only=False, mode='auto', period=1),
            EarlyStopping(monitor='loss', min_delta=0, patience=15, verbose=0, mode='auto'),
            ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10, verbose=VERBOSE, mode='auto',
                              epsilon=0.0001, cooldown=0, min_lr=0)]

In [ ]:
fcn_sm.compile(loss=LOSS, optimizer=OPTIMIZER)

In [ ]:
sm_history = fcn_sm.fit(x=tx_sm,y=ty_sm, epochs=NB_EPOCHS, batch_size=BATCH_SIZE, verbose=VERBOSE, callbacks=CALLBACKS)

In [ ]:
fcn_sm = None

#### Train the network on large cells

In [ ]:
fcn_lg = nn_models.build_fcn_bilinear_8s(nb_classes=1)

In [ ]:
NB_EPOCHS = 1000
OPTIMIZER = Adam(lr=0.001)
LOSS = binary_cross # Binary cross entropy
VERBOSE = 1
BATCH_SIZE = 1

# Create a callback function to save the most accurate version of our model following each epoch, to stop if learning stops, and
# to reduce the learning rate if loss fails to decrease.
CALLBACKS = [ModelCheckpoint('mod/fcn8s_weights_lg.hdf5', monitor='loss', verbose=1, 
                           save_best_only=True, save_weights_only=False, mode='auto', period=1),
            EarlyStopping(monitor='loss', min_delta=0, patience=15, verbose=0, mode='auto'),
            ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10, verbose=VERBOSE, mode='auto',
                              epsilon=0.0001, cooldown=0, min_lr=0)]

In [ ]:
fcn_lg.compile(loss=LOSS, optimizer=OPTIMIZER)

In [ ]:
lg_history = fcn_lg.fit(x=tx_lg,y=ty_lg, epochs=NB_EPOCHS, batch_size=BATCH_SIZE, verbose=VERBOSE, callbacks=CALLBACKS)

In [ ]:
fcn_lg = None

#### Train the network on the full dataset

In [ ]:
fcn_full = nn_models.build_fcn_bilinear_8s(nb_classes=1)

In [ ]:
NB_EPOCHS = 1000
OPTIMIZER = Adam(lr=0.001)
LOSS = binary_cross # Binary cross entropy
VERBOSE = 1
BATCH_SIZE = 1

# Create a callback function to save the most accurate version of our model following each epoch, to stop if learning stops, and
# to reduce the learning rate if loss fails to decrease.
CALLBACKS = [ModelCheckpoint('mod/fcn8s_weights_full.hdf5', monitor='loss', verbose=1, 
                           save_best_only=True, save_weights_only=False, mode='auto', period=1),
            EarlyStopping(monitor='loss', min_delta=0, patience=15, verbose=0, mode='auto'),
            ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10, verbose=VERBOSE, mode='auto',
                              epsilon=0.0001, cooldown=0, min_lr=0)]

In [ ]:
fcn_full.compile(loss=LOSS, optimizer=OPTIMIZER)

In [ ]:
full_history = fcn_full.fit(x=tx_full,y=ty_full, epochs=NB_EPOCHS, batch_size=BATCH_SIZE, verbose=VERBOSE, callbacks=CALLBACKS)

In [ ]:
fcn_full = None

### Segmentation Results Comparison
Results are evaluated through mean IoU comparison.  This will happen by first determining the optimial threshold for each set of data, and then determining the mean IoU for the data on which the network was trained.

In [ ]:
# Generate a set of thresholds to evaluate, between 0 and 1
xRange = np.array(range(100), dtype=np.float32)
xRange += 1
xRange /= 100.

In [ ]:
#Get the small network optimal threshold
fcn_sm = nn_models.build_fcn_bilinear_8s(nb_classes=1)
fcn_sm.load_weights('mod/fcn8s_weights_sm.hdf5')
sm_preds = fcn_sm.predict(vx_sm)

# Get the mean IoU and plot
meanIoU_by_thresh = metrics.meanIoU_thresholds(sm_preds, vy_sm)
fig = plt.figure(figsize=(8, 6))
plt.scatter(xRange, meanIoU_by_thresh)
plt.title('Small Cell Dataset Mean IoU vs. Threshold Value')
plt.ylabel('Mean IoU')
plt.xlabel('Threshold')

# Get the optimal threshold, print the results
sm_thresh = xRange[np.argmax(meanIoU_by_thresh)]
sm_meanIoU = np.amax(meanIoU_by_thresh)
print('Max IoU',np.amax(meanIoU_by_thresh), 'Optimal IoU Threshold',xRange[np.argmax(meanIoU_by_thresh)])
fcn_sm = None

In [ ]:
#Get the large network optimal threshold
fcn_lg = nn_models.build_fcn_bilinear_8s(nb_classes=1)
fcn_lg.load_weights('mod/fcn8s_weights_lg.hdf5')
lg_preds = fcn_lg.predict(vx_lg)

# Get the mean IoU and plot
meanIoU_by_thresh = metrics.meanIoU_thresholds(lg_preds, vy_lg)
fig = plt.figure(figsize=(8, 6))
plt.scatter(xRange, meanIoU_by_thresh)
plt.title('Large Cell Dataset Mean IoU vs. Threshold Value')
plt.ylabel('Mean IoU')
plt.xlabel('Threshold')
lg_thresh = xRange[np.argmax(meanIoU_by_thresh)]
lg_meanIoU = np.amax(meanIoU_by_thresh)
print('Max IoU',np.amax(meanIoU_by_thresh), 'Optimal IoU Threshold',xRange[np.argmax(meanIoU_by_thresh)])
#fcn_lg = None

In [ ]:
#Get the large network optimal threshold
fcn_full = nn_models.build_fcn_bilinear_8s(nb_classes=1)
fcn_full.load_weights('mod/fcn8s_weights_full.hdf5')
full_preds = fcn_full.predict(vx_full)

# Get the mean IoU and plot
meanIoU_by_thresh = metrics.meanIoU_thresholds(full_preds, vy_full)
fig = plt.figure(figsize=(8, 6))
plt.scatter(xRange, meanIoU_by_thresh)
plt.title('Full Cell Dataset Mean IoU vs. Threshold Value')
plt.ylabel('Mean IoU')
plt.xlabel('Threshold')

# Get the optimal threshold, print the results
full_thresh = xRange[np.argmax(meanIoU_by_thresh)]
full_meanIoU = np.amax(meanIoU_by_thresh)
print('Max IoU',np.amax(meanIoU_by_thresh), 'Optimal IoU Threshold',xRange[np.argmax(meanIoU_by_thresh)])
fcn_full = None

#### Compare the mean IoU achieved for each dataset

In [ ]:
meanIoUs = np.array((sm_meanIoU, lg_meanIoU, full_meanIoU))
fig, ax = plt.subplots()
ind = np.arange(1, 4)
sm, lg, full = plt.bar((1, 2, 3), meanIoUs)
plt.title('Mean Intersection Over Union')
sm.set_facecolor('r')
lg.set_facecolor('b')
full.set_facecolor('rebeccapurple')
ax.set_xticks(np.arange(1, 4))
ax.set_xticklabels(['Small Cells', 'Large Cells', 'Combined\nLarge/Small'])
ax.set_ylim([0, 1])

## Count Network Training and Comparison

#### Create 3 datasets
1. The prediction image
2. The predictions with a threshold applied
3. The original image

In [ ]:
# ID the high-performing model, and then train the counters
if (sm_meanIoU > lg_meanIoU) and (sm_meanIoU > full_meanIoU):
    print('Loading small cell blob detector')
    fcn = nn_models.build_fcn_bilinear_8s(nb_classes=1)
    fcn.load_weights('mod/fcn8s_weights_lg.hdf5')
    seg_preds = fcn.predict(tx_lg)
    im_preds = tx_lg
    train_counts = train_counts_lg
    val_counts = val_counts_lg
    thresh = sm_thresh
elif (lg_meanIoU > sm_meanIoU) and (lg_meanIoU > full_meanIoU):
    print('Loading large cell blob detector')
    fcn = nn_models.build_fcn_bilinear_8s(nb_classes=1)
    fcn.load_weights('mod/fcn8s_weights_lg.hdf5')
    seg_preds = fcn.predict(tx_lg)
    im_preds = tx_lg
    train_counts = train_counts_lg
    val_counts = val_counts_lg
    thresh = lg_thresh
else:
    print('Loading large/small combo cell blob detector')
    fcn = nn_models.build_fcn_bilinear_8s(nb_classes=1)
    fcn.load_weights('mod/fcn8s_weights_full.hdf5')
    seg_preds = fcn.predict(tx_full)
    im_preds = tx_full
    train_counts = train_counts_full
    val_counts = val_counts_full
    thresh = full_thresh
print('Threshold',thresh)
plt.imshow(seg_preds[15,:,:,0])

In [ ]:
# Generate a set of thresholds to evaluate, between 0 and 1
xRange = np.array(range(100), dtype=np.float32)
xRange += 1
xRange /= 100.

In [ ]:
    fcn = nn_models.build_fcn_bilinear_8s(nb_classes=1)
    fcn.load_weights('mod/fcn8s_weights_lg.hdf5')
    lg_preds = fcn.predict(vx_lg)
     # Get the mean IoU and plot
    meanIoU_by_thresh = metrics.meanIoU_thresholds(lg_preds, vy_lg)
    fig = plt.figure(figsize=(8, 6))
    plt.scatter(xRange, meanIoU_by_thresh)
    plt.title('Large Cell Dataset Mean IoU vs. Threshold Value')
    plt.ylabel('Mean IoU')
    plt.xlabel('Threshold')
    lg_thresh = xRange[np.argmax(meanIoU_by_thresh)]
    lg_meanIoU = np.amax(meanIoU_by_thresh)
    print('Max IoU',np.amax(meanIoU_by_thresh), 'Optimal IoU Threshold',xRange[np.argmax(meanIoU_by_thresh)])
    seg_preds = fcn.predict(tx_lg)
    im_preds = tx_lg
    train_counts = train_counts_lg
    val_counts = val_counts_lg
    thresh = lg_thresh
    fcn = None

### Process the blob predictions for training the segmentation (blob) counter
Prep the data to be trained using only the blob predictions

In [ ]:
# Form the dataset to have 3 data channels by concatenating the blob predictions
seg_preds = np.concatenate((seg_preds, seg_preds, seg_preds), axis=3)

In [ ]:
seg_preds.shape

In [ ]:
# Apply the optimal threshold to the blobs, convert them to the standard image scale of 255, and then normalize the data
seg_preds_thresh = np.array((seg_preds > thresh) * 255, dtype=np.float32)
avgs = per_chan_avg(arr=seg_preds_thresh)
normalize(arr=seg_preds_thresh, avgs=avgs)

In [ ]:
# Apply the optimal threshold to the blobs without a threshold, convert them to the standard image scale of 255, and then normalize the data
seg_preds_nt = np.array(seg_preds * 255, dtype=np.float32)
avgs_nt = per_chan_avg(arr=seg_preds_nt)
normalize(arr=seg_preds_nt, avgs=avgs_nt)

Use the blob predictions to isolate cells from the original images

In [ ]:
# Get the original large images for our dataset
tx_lg = load_images(img_names=train_meta.filename[480:], file_path=img_path, preprocess='image', target_size=(224,224,3))
ty_lg  = load_images(img_names=train_meta.filename[480:], file_path=gt_path, preprocess='gt', target_size=(224,224))
vx_lg = load_images(img_names=val_meta.filename[120:], file_path=img_path, preprocess='image', target_size=(224,224,3))
vy_lg = load_images(img_names=val_meta.filename[120:], file_path=gt_path, preprocess='gt', target_size=(224,224))

In [ ]:
# Apply the optimal threshold to the blobs, and then use them as masks to isolate cells from the original images
masks = np.array((seg_preds > thresh), dtype=np.float32)
iso_preds = apply_masks(tx_lg, masks)

In [ ]:
# Normalize the masked Data
avgs = per_chan_avg(arr=iso_preds)
normalize(arr=iso_preds, avgs=avgs)

In [ ]:
plt.imshow(iso_preds[100,:,:,0])

## Training the Counting Models

#### Train the segmentation counter

In [ ]:
seg_counter = nn_models.build_count_nn(nb_classes=1)

In [ ]:
CALLBACKS = [ModelCheckpoint('mod/seg_counter.hdf5', monitor='mean_absolute_error', verbose=1, 
                           save_best_only=True, save_weights_only=True, mode='auto', period=1),
            ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10, verbose=1, mode='auto',
                              epsilon=0.0001, cooldown=0, min_lr=0),
            keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=30, verbose=0, mode='auto')]

In [ ]:
seg_counter.compile(optimizer='Adam',
                     loss='mean_squared_error',
                     metrics=['mean_absolute_error'])

In [ ]:
seg_history = seg_counter.fit(seg_preds_thresh, train_counts,
                          epochs=1000,
                          batch_size=40,
                          callbacks=CALLBACKS)

In [ ]:
# Close the trained counter
seg_counter = None

#### Train Segmentation Counter, no threshold

In [ ]:
seg_history = seg_counter.fit(seg_preds, train_counts,
                          epochs=1000,
                          batch_size=40,
                          callbacks=CALLBACKS)

In [ ]:
seg_preds = fcn.predict(seg_preds_nt)

In [ ]:
seg_preds = np.concatenate((seg_preds, seg_preds, seg_preds), axis=3)
# Apply threshold
seg_preds = np.array((seg_preds * 255), dtype=np.float32)
avgs = per_chan_avg(arr=seg_preds)
normalize(arr=seg_preds, avgs=avgs)

In [ ]:
seg_counter_nt = nn_models.build_count_nn(nb_classes=1)

In [ ]:
seg_counter_nt.load_weights('mod/seg_counter.hdf5')

In [ ]:
CALLBACKS = [ModelCheckpoint('mod/seg_counter_nt.hdf5', monitor='mean_absolute_error', verbose=1, 
                           save_best_only=True, save_weights_only=True, mode='auto', period=1),
            ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10, verbose=1, mode='auto',
                              epsilon=0.0001, cooldown=0, min_lr=0),
            keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=30, verbose=0, mode='auto')]

In [ ]:
seg_counter_nt.compile(optimizer='Adam',
                     loss='mean_squared_error',
                     metrics=['mean_absolute_error'])

In [ ]:
seg_history_nt = seg_counter_nt.fit(seg_preds, train_counts,
                          epochs=1000,
                          batch_size=40,
                          callbacks=CALLBACKS)

#### Train the isolated blob image Counter

In [ ]:
img_counter = nn_models.build_count_nn(nb_classes=1)

In [ ]:
img_counter.load_weights('mod/iso_counter.hdf5')

In [ ]:
CALLBACKS = [ModelCheckpoint('mod/iso_counter.hdf5', monitor='mean_absolute_error', verbose=1, 
                           save_best_only=True, save_weights_only=True, mode='auto', period=1),
            ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10, verbose=1, mode='auto',
                              epsilon=0.0001, cooldown=0, min_lr=0),
            keras.callbacks.EarlyStopping(monitor='mean_absolute_error', min_delta=0, patience=30, verbose=0, mode='auto')]

In [ ]:
img_counter.compile(optimizer='Adam',
                     loss='mean_squared_error',
                     metrics=['mean_absolute_error'])

In [ ]:
img_history = img_counter.fit(iso_preds, train_counts,
                          epochs=1000,
                          batch_size=40,
                          callbacks=CALLBACKS)

## Counting Model Evaluation

### Obtain counts from the models for the validation sets and compare

Start with the segmentation (blob detection) network counts

In [ ]:
# Normalize the validation image set
vx_normal = vx_lg
avgs = per_chan_avg(vx_normal)
normalize(arr=vx_normal, avgs=avgs)

In [ ]:
seg_counter = nn_models.build_count_nn(nb_classes=1)

In [ ]:
seg_counter.load_weights('mod/seg_counter.hdf5')

In [ ]:
# Obtain the blob predictions for the validation dataset
seg_vx_lg = fcn.predict(vx_normal)

In [ ]:
seg_vx_lg = np.concatenate((seg_vx_lg,seg_vx_lg,seg_vx_lg), axis=3)
# Apply the optimal threshold to the blobs, convert them to the standard image scale of 255, and then normalize the data
seg_vx_lg = np.array((seg_vx_lg > thresh) * 255, dtype=np.float32)
avgs = per_chan_avg(arr=seg_vx_lg)
normalize(arr=seg_vx_lg, avgs=avgs)

In [ ]:
seg_vx_counts = seg_counter.predict(seg_vx_lg)

In [ ]:
seg_dat = pd.DataFrame({'Count': val_counts,
                   'Pred': seg_vx_counts[:,0]})
#Group the data by count
seg_grouped = seg_dat.groupby('Count')
seg_dat_1sig = seg_grouped.std()
seg_dat_2sig = seg_dat_1sig * 2
seg_dat_mean = seg_grouped.mean()

In [ ]:
seg_dat.to_csv('data/segmented_counts.csv')
seg_grouped.to_csv('data/segmented_counts_grouped.csv')

##### Segmentation counting without threshold
Test the count of the segmentation (blobs) without a threshold applied counting results

In [ ]:
seg_nt_vx_lg = fcn.predict(vx_normal)

In [ ]:
seg_nt_vx_lg = np.concatenate((seg_nt_vx_lg,seg_nt_vx_lg,seg_nt_vx_lg), axis=3)
# Scale the predictions to the pixel scale 255, and then normalize the data
seg_nt_vx_lg = np.array(seg_nt_vx_lg * 255, dtype=np.float32)
avgs = per_chan_avg(arr=seg_nt_vx_lg)
normalize(arr=seg_nt_vx_lg, avgs=avgs)

In [ ]:
seg_counter_nt = nn_models.build_count_nn(nb_classes=1)
seg_counter_nt.load_weights('mod/seg_counter_nt.hdf5')

In [ ]:
seg_nt_vx_counts = seg_counter_nt.predict(seg_nt_vx_lg)

In [ ]:
seg_nt_dat = pd.DataFrame({'Count': val_counts,
                   'Pred': seg_nt_vx_counts[:,0]})
#Group the data by count
seg_nt_grouped = seg_nt_dat.groupby('Count')
seg_nt_dat_1sig = seg_nt_grouped.std()
seg_nt_dat_2sig = seg_nt_dat_1sig * 2
seg_nt_dat_mean = seg_nt_grouped.mean()

In [ ]:
seg_nt_dat.to_csv('data/segmented_nt_counts.csv')

##### Isolated images
Test the results for the isolated blob counts

In [ ]:
fcn = nn_models.build_fcn_bilinear_8s(nb_classes=1)
fcn.load_weights('mod/fcn8s_weights_lg.hdf5')

In [ ]:
iso_vx = fcn.predict(vx_normal)

In [ ]:
iso_vx = np.concatenate((iso_vx,iso_vx,iso_vx), axis=3)
# Apply the optimal threshold to the blobs, and then use them as masks to isolate cells from the original images
masks = np.array((iso_vx > thresh), dtype=np.float32)
iso_preds = apply_masks(vx_lg, masks)

In [ ]:
# Normalize the masked Data
avgs = per_chan_avg(arr=iso_preds)
normalize(arr=iso_preds, avgs=avgs)

In [ ]:
plt.imshow(iso_preds[6,:,:,0])

In [ ]:
img_counter = nn_models.build_count_nn(nb_classes=1)
img_counter.load_weights('mod/iso_counter.hdf5')

In [ ]:
img_vx_counts = img_counter.predict(iso_preds)

In [ ]:
img_dat = pd.DataFrame({'Count': val_counts,
                   'Pred': img_vx_counts[:,0]})
#Group the data by count
img_grouped = img_dat.groupby('Count')
img_dat_1sig = img_grouped.std()
img_dat_2sig = img_dat_1sig * 2
img_dat_mean = img_grouped.mean()

In [ ]:
img_dat.to_csv('data/iso_counts.csv')

## Compare the counting results
Perform the Comparison

In [ ]:
def scatter_with_deviation(val_counts, reg_val_preds, grouped, title):
    plt.figure(figsize=(8,5))
    plt.scatter(val_counts, reg_val_preds, marker=".")
    plt.plot(grouped.mean())
    plt.plot((np.array(range(100)) + 1))
    plt.ylabel('Prediction')
    plt.xlabel('Cell Count')
    plt.title(title)

In [ ]:
scatter_with_deviation(val_counts, seg_vx_counts, seg_grouped, 'Predictions from Segmented Image w/Threshold vs. Actual Counts')

In [ ]:
scatter_with_deviation(val_counts, img_vx_counts, img_grouped,'Predictions from Isolated Blob Images vs. Actual Counts')

In [ ]:
scatter_with_deviation(val_counts, seg_nt_vx_counts, seg_nt_grouped, 'Predictions from Segmented Image w/o Threshold vs. Actual Counts')

In [ ]:
seg_nt_gp = pd.DataFrame(seg_nt_grouped.std())

In [ ]:
plt.plot(seg_nt_gp.index, seg_nt_grouped.std())
plt.scatter(seg_nt_gp.index, seg_grouped.std())
plt.scatter(seg_nt_gp.index, img_grouped.std())
plt.title('Prediction Standard Deviations vs. Actual Counts')

In [ ]:
x = seg_nt_grouped['Count'].all()

In [ ]:
y = x.to_dict()

In [ ]:
# Three subplots sharing both x/y axes
f, (ax1, ax2, ax3) = plt.subplots(3, sharex=True, sharey=True)
ax1.plot(seg_nt_gp.index, seg_nt_grouped.std())
ax1.set_title('Standard Deviation vs. Cell Count')
ax2.plot(seg_nt_gp.index, seg_grouped.std())
ax3.plot(seg_nt_gp.index, img_grouped.std(), color='r')
# Fine-tune figure; make subplots close to each other and hide x ticks for
# all but bottom plot.
f.subplots_adjust(hspace=0)
plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)